In [3]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.99.9.10).
The following additional packages will be installed:
  libgles-dev libgles1 libglew2.1 libglvnd-dev libopengl-dev libosmesa6
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev libglew2.1
  libglvnd-dev libopengl-dev libosmesa6 libosmesa6-dev
0 upgraded, 10 newly installed, 0 to remove and 27 not upgraded.
Need to get 3,438 kB of archives.
After this operation, 17.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1-mesa-glx amd64 21.2.6-0ubuntu0.1~20.04.2 [5,536 B]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles1 amd64 1.3.2-1~ubuntu0.20.04.2 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles-dev amd64 1.3.2-1~ubun

In [4]:
!pip install gym
!pip install free-mujoco-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached free_mujoco_py-2.1.6-py3-none-any.whl (14.1 MB)
  Using cached fasteners-0.15-py2.py3-none-any.whl (23 kB)
  Using cached glfw-1.12.0-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-manylinux2014_x86_64.whl (203 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)


In [5]:
!pip install gym[mujoco]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


In [10]:
import mujoco_py
import gym

In [12]:
env = gym.make("HalfCheetah-v2")

In [8]:
import tensorflow as tf
from tensorflow.keras import * 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_probability as tfp

In [15]:
print(env.reset().shape)
print(env.action_space.high), print(env.action_space.low)

(17,)
[1. 1. 1. 1. 1. 1.]
[-1. -1. -1. -1. -1. -1.]


(None, None)

In [16]:
print(env.action_space.shape)

(6,)


In [17]:
env.step([1., 1., 1., 1., 1., 1.])

(array([ 0.04984378, -0.12455043,  0.25254534,  0.27222136,  0.35601046,
         0.27977517,  0.21869659,  0.18465768,  1.45686154, -1.13167554,
        -3.0367347 ,  8.70556711,  8.89775993,  7.45081049,  9.55547678,
         7.80910497,  7.47987164]),
 0.4710522792822871,
 False,
 {'reward_run': 1.0710522792822872, 'reward_ctrl': -0.6000000000000001})

In [18]:
class ReplayBuffer:
    
    def __init__(self, max_size, batch_size):
        
        self.buffer = []
        self.max_size = max_size
        self.batch_size = batch_size
    
    def append(self, state, next_state, action, reward, done, info):
        
        sample = (state, next_state, action, reward, done, info)
        
        if len(self.buffer) >= self.max_size:
            
            self.buffer.pop(0)
        
        self.buffer.append(sample)
        
    def sample(self):
        
        idxs = np.random.choice(np.arange(len(self.buffer)), size = self.batch_size)
        
        states = np.array([self.buffer[i][0] for i in idxs])
        next_states = np.array([self.buffer[i][1] for i in idxs])
        actions = np.array([self.buffer[i][2] for i in idxs])
        rewards = np.array([self.buffer[i][3] for i in idxs])
        dones = np.array([self.buffer[i][4] for i in idxs])
            
        return states, next_states, actions, rewards, dones

In [19]:
class ValueNetwork(tf.keras.Model):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNetwork, self).__init__()
        
        self.linear1 = layers.Dense(hidden_dim, activation='relu', input_shape=(None, state_dim))
        self.linear2 = layers.Dense(hidden_dim, activation='relu')
        self.linear3 = layers.Dense(1)

        
    def call(self, state):
        x = self.linear1(state)
        x = self.linear2(x)
        x = self.linear3(x)
        return x

In [20]:
class SoftCritic(tf.keras.Model):
    def __init__(self, num_inputs, num_actions, hidden_size):
        super(SoftCritic, self).__init__()
        
        self.linear1 = tf.keras.layers.Dense(hidden_size, input_shape=(None, num_inputs + num_actions), activation='relu')
        self.linear2 = tf.keras.layers.Dense(hidden_size, activation='relu')
        self.linear3 = tf.keras.layers.Dense(1)
        
        
    def call(self, state, action):
        x = tf.concat([state, action], axis=1)
        x = tf.keras.layers.ReLU()(self.linear1(x))
        x = tf.keras.layers.ReLU()(self.linear2(x))
        x = self.linear3(x)
        return x

In [29]:
class Actor(tf.keras.Model):
    def __init__(self, state_dim, num_actions, hidden_size, action_high=-1, action_low=1):
        super(Actor, self).__init__()
        
        self.action_high = action_high
        self.action_low = action_low
        
        self.linear1 = layers.Dense(hidden_size, activation='relu', input_shape=(None, state_dim))
        self.linear2 = layers.Dense(hidden_size, activation='relu')
        
        self.mean_linear = layers.Dense(num_actions)
        self.log_std_linear = layers.Dense(num_actions)
        
    def call(self, state):
        x = self.linear1(state)
        x = self.linear2(x)
        
        mean    = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = tf.clip_by_value(log_std, self.action_low, self.action_high)
        
        return mean, log_std
    
    def evaluate(self, state, epsilon=1e-6):
        mean, log_std = self.call(state)
        std = tf.exp(log_std)
        
        normal = tfp.distributions.Normal(mean, std)
        z = normal.sample()
        action = tf.tanh(z)
        
        log_prob = normal.log_prob(z) - tf.math.log(1 - tf.pow(action, 2) + epsilon)
        log_prob = tf.reduce_sum(log_prob, axis=-1, keepdims=True)
        
        return action, log_prob, z, mean, log_std
        
    def get_action(self, state):
        mean, log_std = self.call(tf.expand_dims(state, axis=0))
        std = tf.exp(log_std)
        
        normal = tfp.distributions.Normal(mean, std)
        z      = normal.sample()
        action = tf.tanh(z)
        
        return action.numpy()[0]

In [68]:
class SAC:
    
    def __init__(self, env, value_net, value_target, dqn, dqn2, policy_net, replay_buffer, gamma, learning_rate):
        
        self.env = env
        self.dqn = dqn
        self.dqn2 = dqn2
        self.value_net = value_net
        self.value_target = value_target
        self.policy = policy_net
        self.experience_replay = replay_buffer
        self.counter = 0
        self.update_rate = 80
        self.gamma = gamma
        self.value_net_opt = optimizers.Adam(learning_rate=learning_rate)
        self.critic_1_opt = optimizers.Adam(learning_rate=learning_rate)
        self.critic_2_opt = optimizers.Adam(learning_rate=learning_rate)
        self.policy_opt = optimizers.Adam(learning_rate=learning_rate)


        

    def train(self):

      if len(self.experience_replay.buffer) < self.experience_replay.batch_size:

        return

      states, next_states, actions, rewards, dones = self.experience_replay.sample()

      #Training Critic
      with tf.GradientTape() as tape:
        with tf.GradientTape() as tape2:
          q_values1 = self.dqn(states, actions)
          q_values2 = self.dqn2(states, actions)
          target_value = self.value_target(states)

          target_q_value = rewards.reshape(-1,1) + (1-dones.reshape(-1,1))*self.gamma*target_value

          loss1 = tf.reduce_mean(tf.square(q_values1 - target_q_value))
          loss2 = tf.reduce_mean(tf.square(q_values2 - target_q_value))

      # Obtener los gradientes
      grads1 = tape.gradient(loss1, self.dqn.trainable_variables)
      grads2 = tape2.gradient(loss2, self.dqn2.trainable_variables)

      # Actualizar los parámetros
      self.critic_1_opt.apply_gradients(zip(grads1, self.dqn.trainable_variables))
      self.critic_2_opt.apply_gradients(zip(grads2, self.dqn2.trainable_variables))

      #Training Value Net

      new_actions, log_probs, z, mean, log_std = self.policy.evaluate(states)

      with tf.GradientTape() as tape:

        prior_values = self.value_net(states)
        q_values1 = self.dqn(states, new_actions)
        q_values2 = self.dqn2(states, new_actions)
        if tf.math.reduce_mean(q_values1) >= tf.math.reduce_mean(q_values2):
          q_value = q_values1 
        else:
          q_value = q_values2

        target_values = q_value - log_probs

        value_loss = tf.math.reduce_mean(tf.square(prior_values-target_values))
      
      grads_value = tape.gradient(value_loss, self.value_net.trainable_variables)

      self.value_net_opt.apply_gradients(zip(grads_value,self.value_net.trainable_variables))

      #Training Policy

      with tf.GradientTape() as tape:

        new_actions, log_probs, z, mean, log_std = self.policy.evaluate(states)

        q_values1 = self.dqn(states, new_actions)
        q_values2 = self.dqn2(states, new_actions)
        if tf.math.reduce_mean(q_values1) >= tf.math.reduce_mean(q_values2):
          q_value = q_values1 
        else:
          q_value = q_values2
        policy_loss = tf.reduce_mean(log_probs - q_value)

      grads_policy = tape.gradient(policy_loss, self.policy.trainable_variables)
      self.policy_opt.apply_gradients(zip(grads_policy, self.policy.trainable_variables))

      self.value_target.set_weights(self.value_net.get_weights())



In [69]:
value_net = ValueNetwork(17, 32)
value_target = ValueNetwork(17, 32)
value_target.set_weights(value_net.get_weights())

critic_1 = SoftCritic(17, 6, 32)
critic_2 = SoftCritic(17, 6, 32)

actor = Actor(17, 6, 32)

experience_replay = ReplayBuffer(100000, 64)

In [70]:
gamma = 0.99
episodes = 200 
learning_rate = 3e-4


value_net_opt = optimizers.Adam(learning_rate=learning_rate)
critic_1_opt =  optimizers.Adam(learning_rate=learning_rate)
critic_2_opt =  optimizers.Adam(learning_rate=learning_rate)

policy_opt = optimizers.Adam(learning_rate=learning_rate)

In [81]:
sac = SAC(env, value_net, value_target, critic_1, critic_2, actor, experience_replay, gamma, learning_rate)

In [82]:
returns = []
avg_returns = []
total_steps = 0

In [ ]:
for episode in range(episodes):

  Return = 0

  done = False
  state = env.reset()
  steps = 0

  while not done:

    action = sac.policy.get_action(state)
    next_state, reward, done, info = env.step(action)
    Return += reward
    experience_replay.append(state, next_state, action, reward, done, info)
    state = next_state
    steps += 1

  sac.train()

  returns.append(Return)
  avg_return = np.mean(returns[-10:])
  avg_returns.append(avg_return)

  total_steps += steps

  print("Episode: " + str(episode)+"/"+str(episodes) + " return of "+ str(Return) + " average reward of: " + str(avg_return) +  "in " + str(steps) + " steps")

  if episode % 10 == 0:
    sac.dqn.save_weights(f"critic1_{ episode }", save_format='tf')
    sac.dqn2.save_weights(f"critic2_{ episode }", save_format='tf')
    
    sac.value_net.save_weights(f"value_{ episode }", save_format='tf')

    sac.policy.save_weights(f"actor_{ episode }", save_format='tf')

Episode: 0/200 return of -544.08041437002 average reward of: -544.08041437002in 1000 steps
Episode: 1/200 return of -388.20363917574366 average reward of: -466.1420267728818in 1000 steps
Episode: 2/200 return of -409.32331225746003 average reward of: -447.20245526774124in 1000 steps
Episode: 3/200 return of -414.97393003577827 average reward of: -439.1453239597505in 1000 steps
Episode: 4/200 return of -248.06427721751615 average reward of: -400.9291146113036in 1000 steps
Episode: 5/200 return of -272.0694809060938 average reward of: -379.4525089937686in 1000 steps
Episode: 6/200 return of -305.5794577183668 average reward of: -368.89921595442553in 1000 steps
Episode: 7/200 return of -457.70948336774114 average reward of: -380.00049938109in 1000 steps
Episode: 8/200 return of -439.8349453607148 average reward of: -386.64877115660386in 1000 steps
Episode: 9/200 return of -433.1538151185781 average reward of: -391.2992755528013in 1000 steps
Episode: 10/200 return of -580.2035011587872 ave